In [1]:
# SOURCE: https://colab.research.google.com/drive/1AmcITfN2ELQe07WKQ9szaxq-WSu4hdQb#scrollTo=kLGQdA1x66gA
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import nemo
from tqdm import tqdm

In [2]:
from mobnet import ModelMobnet


In [3]:
# convenience class to keep track of averages
class Metric(object):
    def __init__(self, name):
        self.name = name
        self.sum = torch.tensor(0.)
        self.n = torch.tensor(0.)
    def update(self, val):
        self.sum += val.cpu()
        self.n += 1
    @property
    def avg(self):
        return self.sum / self.n
    
def test(model, device, test_loader, verbose=True):
    model.eval()
    test_loss = 0
    correct = 0
    test_acc = Metric('test_acc')
    with tqdm(total=len(test_loader),
          desc='Test',
          disable=not verbose) as t:
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
                pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()
                test_acc.update((pred == target.view_as(pred)).float().mean())
                t.set_postfix({'acc' : test_acc.avg.item() * 100. })
                t.update(1)
    test_loss /= len(test_loader.dataset)
    return test_acc.avg.item() * 100.

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor()
    ])),
    batch_size=128, shuffle=False, **kwargs
)

In [5]:
!rm -rf mnist_cnn_fp.pt
!wget https://raw.githubusercontent.com/FrancescoConti/nemo_examples_helper/master/mnist_cnn_fp.pt
!md5sum mnist_cnn_fp.pt
model = ExampleNet().to(device)
state_dict = torch.load("mnist_cnn_fp.pt", map_location='cpu')
model.load_state_dict(state_dict, strict=True)
acc = test(model, device, test_loader)
print("\nFullPrecision accuracy: %.02f%%" % acc)

--2022-02-09 18:05:19--  https://raw.githubusercontent.com/FrancescoConti/nemo_examples_helper/master/mnist_cnn_fp.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9527849 (9,1M) [application/octet-stream]
Saving to: ‘mnist_cnn_fp.pt’

mnist_cnn_fp.pt     100%[===================>]   9,09M  1,99MB/s    in 5,3s    

2022-02-09 18:05:24 (1,72 MB/s) - ‘mnist_cnn_fp.pt’ saved [9527849/9527849]

40833b928b5179255cdde541ac97f58c  mnist_cnn_fp.pt


Test: 100%|███████████████████████████| 79/79 [00:09<00:00,  8.59it/s, acc=99.1]


FullPrecision accuracy: 99.12%


In [6]:
model = nemo.transform.quantize_pact(model, dummy_input=torch.randn((1,1,28,28)).to(device))
precision = {
    'conv1': {
        'W_bits' : 15
    },
    'conv2': {
        'W_bits' : 15
    },
    'fc1': {
        'W_bits' : 15
    },
    'fc2': {
        'W_bits' : 15
    },
    'relu1': {
        'x_bits' : 16
    },
    'relu2': {
        'x_bits' : 16
    },
    'fcrelu1': {
        'x_bits' : 16
    },
}
model.change_precision(bits=1, min_prec_dict=precision)
acc = test(model, device, test_loader)
print("\nFakeQuantized @ 16b accuracy (first try): %.02f%%" % acc)

Test: 100%|███████████████████████████| 79/79 [00:11<00:00,  6.61it/s, acc=82.2]


FakeQuantized @ 16b accuracy (first try): 82.21%


In [7]:
with model.statistics_act():
    _ = test(model, device, test_loader)
model.reset_alpha_act()
acc = test(model, device, test_loader)
print("\nFakeQuantized @ 16b accuracy (calibrated): %.02f%%" % acc)

Test: 100%|███████████████████████████| 79/79 [00:18<00:00,  4.38it/s, acc=99.1]


FakeQuantized @ 16b accuracy (calibrated): 99.12%


In [8]:
precision = {
    'conv1': {
        'W_bits' : 7
    },
    'conv2': {
        'W_bits' : 7
    },
    'fc1': {
        'W_bits' : 7
    },
    'fc2': {
        'W_bits' : 3
    },
    'relu1': {
        'x_bits' : 8
    },
    'relu2': {
        'x_bits' : 8
    },
    'fcrelu1': {
        'x_bits' : 8
    },
}
model.change_precision(bits=1, min_prec_dict=precision)
acc = test(model, device, test_loader)
print("\nFakeQuantized @ mixed-precision accuracy: %.02f%%" % acc)

Test: 100%|███████████████████████████| 79/79 [00:39<00:00,  1.98it/s, acc=99.1]


FakeQuantized @ mixed-precision accuracy: 99.10%


In [9]:
# save checkpoint using NEMO's dedicated function
nemo.utils.save_checkpoint(model, None, 0, checkpoint_name='mnist_fq_mixed')
# load it back (just for fun!) with PyTorch's one
checkpoint = torch.load('checkpoint/mnist_fq_mixed.pth')
# pretty-print the precision dictionary
import json
print(json.dumps(checkpoint['precision'], indent=2))

{
  "conv1": {
    "W_bits": 7
  },
  "relu1": {
    "x_bits": 8
  },
  "conv2": {
    "W_bits": 7
  },
  "relu2": {
    "x_bits": 8
  },
  "fc1": {
    "W_bits": 7
  },
  "fcrelu1": {
    "x_bits": 8
  },
  "fc2": {
    "W_bits": 7
  }
}


In [10]:
model.fold_bn()
model.reset_alpha_weights()
acc = test(model, device, test_loader)
print("\nFakeQuantized @ mixed-precision (folded) accuracy: %.02f%%" % acc)

Test: 100%|███████████████████████████| 79/79 [00:11<00:00,  6.68it/s, acc=99.1]


FakeQuantized @ mixed-precision (folded) accuracy: 99.11%


In [11]:
model = nemo.transform.bn_to_identity(model) # necessary because folding does not physically remove BN layers
model.qd_stage(eps_in=1./255)
print(model)
acc = test(model, device, test_loader)
print("\nQuantizedDeployable @ mixed-precision accuracy: %.02f%%" % acc)

ExampleNet(
  (conv1): PACT_Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn1): PACT_Identity()
  (relu1): PACT_Act()
  (conv2): PACT_Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn2): PACT_Identity()
  (relu2): PACT_Act()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): PACT_Linear(in_features=9216, out_features=256, bias=True)
  (fcrelu1): PACT_Act()
  (fc2): PACT_Linear(in_features=256, out_features=10, bias=True)
)


Test: 100%|███████████████████████████| 79/79 [00:22<00:00,  3.54it/s, acc=99.1]


QuantizedDeployable @ mixed-precision accuracy: 99.10%


In [ ]:
model.id_stage()
print(model)
acc = test(model, device, test_loader)
print("\nIntegerDeployable @ mixed-precision accuracy: %.02f%%" % acc)

ExampleNet(
  (conv1): PACT_Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn1): PACT_Identity()
  (relu1): PACT_IntegerAct()
  (conv2): PACT_Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn2): PACT_Identity()
  (relu2): PACT_IntegerAct()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): PACT_Linear(in_features=9216, out_features=256, bias=True)
  (fcrelu1): PACT_IntegerAct()
  (fc2): PACT_Linear(in_features=256, out_features=10, bias=True)
)


Test:  33%|████████▉                  | 26/79 [00:04<00:08,  6.18it/s, acc=11.4]

In [ ]:
def test_with_integer(model, device, test_loader, verbose=True, integer=False):
    model.eval()
    test_loss = 0
    correct = 0
    test_acc = Metric('test_acc')
    with tqdm(total=len(test_loader),
          desc='Test',
          disable=not verbose) as t:
        with torch.no_grad():
            for data, target in test_loader:
                if integer:      # <== this is different from the previous version
                    data *= 255  # <== of test function!
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
                pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()
                test_acc.update((pred == target.view_as(pred)).float().mean())
                t.set_postfix({'acc' : test_acc.avg.item() * 100. })
                t.update(1)
    test_loss /= len(test_loader.dataset)
    return test_acc.avg.item() * 100.

In [ ]:
acc = test_with_integer(model, device, test_loader, integer=True)
print("\nIntegerDeployable @ mixed-precision accuracy (for real): %.02f%%" % acc)

In [ ]:
nemo.utils.export_onnx('mnist_id_mixed.onnx', model, model, (1,28,28))